In [1]:
# autoreload
%load_ext autoreload
%autoreload 1
# write code that works in both Python 2 (2.7+) and 3 (3.5+)
from __future__ import absolute_import, division, print_function, unicode_literals
try:
    input = raw_input
    range = xrange
except NameError:
    pass

import csv
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

import keras
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential

import pprint
pp = pprint.PrettyPrinter(indent=2).pprint
from IPython.display import display as dd
from IPython.display import Image

%matplotlib inline

%aimport lib.helpers
%aimport lib.etl
%aimport lib.visualization
%aimport lib.walk_forward_predict
%aimport models.rnn
%aimport models.lr
from lib.helpers import (
    timing, print_shapes, fix_all_rngs,
    date_str_to_ts
)
from lib.visualization import simple_plot
from lib.walk_forward_predict import (
    run_walk_forward_validation_rnn,
    run_walk_forward_validation_arima,
    run_walk_forward_validation_rnn_retraining
)
import lib.etl as etl
from models.rnn import make_rnn_model_constructor
from models.lr import LRModel

# uncomment when acttually working on this, it's here for reproducibility though
fix_all_rngs(42) 

Using TensorFlow backend.


In [2]:
RESULTS_FILE = './full_results/btc5min_ohlc.csv'

r = pd.read_csv(RESULTS_FILE)
dd(r)
print("> DACC: %.2f%%" % r['dir_acc'].str.replace('%', '').astype(float).mean())
print("> RMSE: %.4f, vs. %.4f for CP" % (
    r['rmse'].str.replace('%', '').astype(float).mean(),
    r['rmse_cp'].str.replace('%', '').astype(float).mean(),
))

,model_description,train_from,train_to,test_on,shuffle,normalize,training_loss,rmse,rmse_cp,dir_acc
0,"RNN(100|0.1|50|0.1|linear, RMSprop, mse) close...",0,25000,3000,False,seqs,0.001194,0.0379%,0.0162%,44.2383%
1,"RNN(100|0.1|50|0.1|linear, RMSprop, mse) close...",3000,28000,3000,False,seqs,0.000832,0.0563%,0.0134%,41.7480%
2,"RNN(100|0.1|50|0.1|linear, RMSprop, mse) close...",6000,31000,3000,False,seqs,0.000717,0.0216%,0.0091%,52.2949%
3,"RNN(100|0.1|50|0.1|linear, RMSprop, mse) close...",9000,34000,3000,False,seqs,0.000728,0.0131%,0.0110%,45.8008%
4,"RNN(100|0.1|50|0.1|linear, RMSprop, mse) close...",12000,37000,3000,False,seqs,0.000371,0.0284%,0.0099%,56.2988%
5,"RNN(100|0.1|50|0.1|linear, RMSprop, mse) close...",15000,40000,3000,False,seqs,0.000452,0.0165%,0.0111%,50.8789%
6,"RNN(100|0.1|50|0.1|linear, RMSprop, mse) close...",18000,43000,3000,False,seqs,0.001008,0.0254%,0.0197%,43.0664%
7,"RNN(100|0.1|50|0.1|linear, RMSprop, mse) close...",21000,46000,3000,False,seqs,0.000716,0.0342%,0.0312%,48.4863%
8,"RNN(100|0.1|50|0.1|linear, RMSprop, mse) close...",24000,49000,3000,False,seqs,0.000819,0.0675%,0.0142%,58.5449%
9,"RNN(100|0.1|50|0.1|linear, RMSprop, mse) close...",27000,52000,3000,False,seqs,0.000845,0.0162%,0.0110%,36.7676%


> DACC: 48.08%
> RMSE: 0.0395, vs. 0.0235 for CP
